In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear

import torch_geometric.nn as pyg_nn
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(precision=5, suppress=True)
torch.set_printoptions(precision=5, sci_mode=False)


In [3]:
dataset1 = pd.read_csv('dataset/measured_active_power.csv')
dataset2 = pd.read_csv('dataset/measured_reactive_power.csv')
dataset3 = pd.read_csv('dataset/actual_voltage_angles.csv')
dataset4 = pd.read_csv('dataset/actual_voltage_magnitudes.csv')

In [4]:
df1 = pd.DataFrame(dataset1)
df2 = pd.DataFrame(dataset2)
df3 = pd.DataFrame(dataset3)
df4 = pd.DataFrame(dataset4)

In [5]:
df1 = df1.drop(columns=['Timestep'])
df2 = df2.drop(columns=['Timestep'])
df3 = df3.drop(columns=['Timestep'])
df4 = df4.drop(columns=['Timestep'])

phase1_df_p = df1.filter(regex='.1$')
phase1_df_q = df2.filter(regex='.1$')
phase1_df_v = df3.filter(regex='.1$')
phase1_df_d = df4.filter(regex='.1$')

In [6]:
file_path = r'C:\FREEDM\gnn-powerflow\jupyter notebook\dataset\edge_matrix.csv'

df = pd.read_csv(file_path, index_col=0)

In [7]:
#function to sort the column issues

def update_dataframe_columns(df, column_list):
    # Step 1: Extract the part of the column names before `.1`
    cleaned_columns = df.columns.str.replace(r'\.\d+', '', regex=True).str.lower()
    
    # Step 2: Compare with the provided list of columns (ignore case)
    column_list_lower = [col.lower() for col in column_list]
    
    # Step 3: Find missing columns from the provided list
    missing_columns = [col for col in column_list_lower if col not in cleaned_columns]
    
    # Step 4: Add missing columns with the `.1` suffix and initialize them with 0
    for missing_col in missing_columns:
        df[f'{missing_col}.1'] = 0
    
    # Step 5: Return the DataFrame with all column names in lowercase
    return df.rename(columns=str.lower)

In [8]:
def reorder_columns(df, column_list):
    # Step 1: Strip the `.1` suffix from the DataFrame columns
    ordered_cols = [f"{col}.1" for col in column_list]

    return df[ordered_cols]

In [9]:
desired_cols = df.columns

In [10]:
phase1_df_p_cleaned = update_dataframe_columns(phase1_df_p, desired_cols)
p1_p = reorder_columns(phase1_df_p_cleaned, desired_cols)

phase1_df_q_cleaned = update_dataframe_columns(phase1_df_q, desired_cols)
p1_q = reorder_columns(phase1_df_q_cleaned, desired_cols)

phase1_df_v_cleaned = update_dataframe_columns(phase1_df_v, desired_cols)
p1_v = reorder_columns(phase1_df_v_cleaned, df.columns)

phase1_df_d_cleaned = update_dataframe_columns(phase1_df_d, desired_cols)
p1_d = reorder_columns(phase1_df_d_cleaned, desired_cols)

C:\Users\panka\AppData\Local\Temp\ipykernel_39868\701338628.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{missing_col}.1'] = 0
C:\Users\panka\AppData\Local\Temp\ipykernel_39868\701338628.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{missing_col}.1'] = 0
C:\Users\panka\AppData\Local\Temp\ipykernel_39868\701338628.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [11]:
p1_p.columns = ['P_' + str(col) for col in p1_p.columns]
p1_q.columns = ['Q_' + str(col) for col in p1_q.columns]
p1_v.columns = ['V_' + str(col) for col in p1_v.columns]
p1_d.columns = ['d_' + str(col) for col in p1_d.columns]

In [12]:
combined_df = pd.concat([p1_p, p1_q, p1_v, p1_d], axis=1)

# Sorting columns by the suffix part to ensure same suffix columns are next to each other
combined_df = combined_df.reindex(sorted(combined_df.columns, key=lambda x: x.split('_')[1]), axis=1)

In [13]:
def slice_dataset(dataset, percentage):
    data_size = len(dataset)
    return dataset[:int(data_size*percentage/100)]

def make_dataset(dataset, n_bus):
    x_raw_1, y_raw_1 = [], []
    x_raw, y_raw = [], []

    for i in range(len(dataset)):
        for n in range(n_bus):
            x_raw_1.append(list([dataset[i, 4*n], dataset[i, 4*n+2]]))
            y_raw_1.extend(dataset[i, 4*n+2:4*n+4])

        x_raw.append(list(x_raw_1))
        y_raw.append(y_raw_1)
        x_raw_1, y_raw_1 = [], []

    x_raw = torch.tensor(x_raw, dtype=torch.float)
    y_raw = torch.tensor(y_raw, dtype=torch.float)
    return x_raw, y_raw

def normalize_dataset(x, y):
    x_mean = torch.mean(x,0)
    y_mean = torch.mean(y,0)
    x_std = torch.std(x,0)
    y_std = torch.std(y,0)
    x_norm = (x-x_mean)/x_std
    y_norm = (y-y_mean)/y_std
    x_norm = torch.where(torch.isnan(x_norm), torch.zeros_like(x_norm), x_norm)
    y_norm = torch.where(torch.isnan(y_norm), torch.zeros_like(y_norm), y_norm)
    x_norm = torch.where(torch.isinf(x_norm), torch.zeros_like(x_norm), x_norm)
    y_norm = torch.where(torch.isinf(y_norm), torch.zeros_like(y_norm), y_norm)
    return x_norm, y_norm, x_mean, y_mean, x_std, y_std

def denormalize_output(y_norm, y_mean, y_std):
    y = y_norm*y_std+y_mean
    return y

def NRMSE(yhat,y):
    return torch.sqrt(torch.mean(((yhat-y)/torch.std(yhat,0))**2))

def MSE(yhat,y):
    return torch.mean((yhat-y)**2)

In [26]:
class My_GNN_NN(torch.nn.Module):
    def __init__(self, node_size=None, feat_in=None, feat_size1=None, hidden_size1=None, output_size=None):
        super(My_GNN_NN, self).__init__()
        self.feat_in = feat_in if feat_in is not None else 2
        self.feat_size1 = feat_in if feat_in is not None else 4
        self.hidden_size1 = hidden_size1 if hidden_size1 is not None else 20
        self.output_size = output_size if output_size is not None else 12
        
        self.conv1 = GCNConv(feat_in, feat_size1)
        self.lin1 = Linear(node_size*feat_size1, hidden_size1)
        self.lin2 = Linear(hidden_size1, output_size)

    def forward(self, data):
        
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.tanh(x)

        x = x.flatten(start_dim = 0)
        x = self.lin1(x)
        x = torch.tanh(x)

        x = self.lin2(x)

        return x
    
    def save_weights(self, model, name):
        torch.save(model, name)

In [15]:
percentage = 100
n_bus_p1 = 84
n_bus_p2 = 84
n_bus_p3 = 84

#Phase1
phase1_dataset = slice_dataset(combined_df, percentage).to_numpy()
x_raw_phase1, y_raw_phase1 = make_dataset(phase1_dataset, n_bus_p1)

In [16]:
from sklearn.model_selection import train_test_split

# Assuming X is your features and y is your labels
X_train_p1, X_temp_p1, y_train_p1, y_temp_p1 = train_test_split(x_raw_phase1, y_raw_phase1, test_size=0.5, random_state=42)

# Now split X_temp and y_temp into validation and test sets
X_val_p1, X_test_p1, y_val_p1, y_test_p1 = train_test_split(X_temp_p1, y_temp_p1, test_size=0.5, random_state=42)

#Phase1

x_norm_phase1_train, y_norm_phase1_train, _, _, _, _ = normalize_dataset(X_train_p1, y_train_p1)
x_norm_phase1_val, y_norm_phase1_val, x_phase1_val_mean, y_phase1_val_mean, x_phase1_val_std, y_phase1_val_std = normalize_dataset(X_val_p1, y_val_p1)

In [17]:
# Step 2: Extract row and column names (node names) from the CSV (they should be the same in a matrix)
row_names = df.index.tolist()    # Row names (from the first column)
col_names = df.columns.tolist()  # Column names (from the first row)

# Ensure that the row and column names match (adjacency matrix should be square)
assert row_names == col_names, "Row names and column names must match in an adjacency matrix."

# Step 3: Initialize two lists to store source and target nodes
source_nodes = []
target_nodes = []

# Step 4: Iterate through the DataFrame to extract edges (ignore the diagonal)
for i in range(df.shape[0]):  # iterate over rows (source nodes)
    for j in range(df.shape[1]):  # iterate over columns (target nodes)
        if df.iloc[i, j] == 1:  # if there's an edge between node i and node j
            source_nodes.append(row_names[i])  # append the source node name
            target_nodes.append(col_names[j])  # append the target node name

# Step 5: Convert node names to numerical indices
node_to_index = {name: idx for idx, name in enumerate(row_names)}

source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

# Step 6: Create the edge_index tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [18]:
# Assuming x_train, y_train, x_val, y_val are already defined
x_train, y_train = x_norm_phase1_train, y_norm_phase1_train
x_val, y_val = x_norm_phase1_val, y_norm_phase1_val

data_train_list, data_val_list = [], []

# Generate Data for training
for i in range(len(x_train)):
    n_nodes = x_train[i].shape[0]  # Determine the number of nodes dynamically
   
    data_train_list.append(Data(x=x_train[i], y=y_train[i], edge_index=edge_index))

# Generate Data for validation
for i in range(len(x_val)):
    n_nodes = x_val[i].shape[0]  # Determine the number of nodes dynamically
    
    data_val_list.append(Data(x=x_val[i], y=y_val[i], edge_index=edge_index))

# DataLoaders
train_loader = DataLoader(data_train_list, batch_size=1)
val_loader = DataLoader(data_val_list, batch_size=1)

c:\Users\panka\.virtualenvs\gnn-powerflow-0Gf6W7zT\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [19]:
n_bus_p1 = 84
feat_in = 2
feat_size1 = 4
hidden_size1 = 168
output_size = n_bus_p1*2
lr = 0.0001

model = My_GNN_NN(n_bus_p1, feat_in, feat_size1, hidden_size1, output_size)
for name, param in model.named_parameters():
  print(name)
  print(param.size())

param = sum(p.numel() for p in model.parameters() if p.requires_grad)

conv1.bias
torch.Size([4])
conv1.lin.weight
torch.Size([4, 2])
lin1.weight
torch.Size([168, 336])
lin1.bias
torch.Size([168])
lin2.weight
torch.Size([168, 168])
lin2.bias
torch.Size([168])


In [20]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [21]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [28]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your parameters
feat_in = 2
feat_size1 = 4
hidden_size1 = 84
output_size = n_bus_p1 * 2  # Ensure n_bus_p1 is defined in your code
lr = 0.0001

# Initialize the model and move it to GPU if available
model = My_GNN_NN(n_bus_p1, feat_in, feat_size1, hidden_size1, output_size).to(device)

# Optimizer remains the same
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Lists for storing training and validation losses
train_loss_list, val_loss_list = [], []

# Early stopping parameters
count = 0
patience = 2000
lossMin = 1e10

# Set the number of steps to accumulate gradients over
accumulation_steps = 4  # This means accumulate gradients over 4 batches before updating weights

# Main training loop
for epoch in range(2001):
    model.train()  # Set model to training mode
    train_loss = 0

    for batch in train_loader:
        # Move batch and all required tensors to GPU
        batch = batch.to(device)  # Make sure batch data is on the GPU
        optimizer.zero_grad()

        # Forward pass
        y_train_prediction = model(batch)

        # Ensure target is on the same device
        target = batch.y.to(device)

        # Compute loss
        loss = MSE(
            denormalize_output(y_train_prediction, y_phase1_val_mean.to(device), y_phase1_val_std.to(device)),
            denormalize_output(target, y_phase1_val_mean.to(device), y_phase1_val_std.to(device))
        )

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        train_loss += loss.item() * batch.num_graphs

    train_loss /= len(train_loader.dataset)
    train_loss_list.append(train_loss)


    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Disable gradient computation for validation
        for batch in val_loader:
            # Move the batch data to the same device as the model (GPU or CPU)
            batch = batch.to(device)

            # Forward pass
            y_val_prediction = model(batch)

            # Compute validation loss
            loss = MSE(
                denormalize_output(y_val_prediction, y_phase1_val_mean.to(device), y_phase1_val_std.to(device)),
                denormalize_output(batch.y, y_phase1_val_mean.to(device), y_phase1_val_std.to(device))
            )

            # Accumulate the loss, weighted by the number of graphs
            val_loss += loss.item() * batch.num_graphs

    # Average validation loss over the dataset
    val_loss /= len(val_loader.dataset)
    val_loss_list.append(val_loss)

    # Early stopping mechanism
    if val_loss < lossMin:
        lossMin = val_loss
        count = 0
        best_epoch = epoch
        best_train_loss = train_loss
        best_val_loss = val_loss
        
        # Save the best model
        model.save_weights(model, "New_GNN_NN.pt")
    else:
        count += 1
        if count > patience:
            print(f"Early stop at epoch {epoch}    train loss: {train_loss:.7f}    val loss: {val_loss:.7f}")
            print(f"Best val at epoch {best_epoch}    train loss: {best_train_loss:.7f}    val loss: {best_val_loss:.7f}")
            break

    # Stop if training loss is too low
    if train_loss <= 0:
        print(f"Min train loss at epoch {epoch}    train loss: {train_loss:.7f}    val loss: {val_loss:.7f}")
        break

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}    train loss: {train_loss:.7f}    val loss: {val_loss:.7f}")


Epoch: 0    train loss: 14243.8764695    val loss: 2434.6410006


KeyboardInterrupt: 

In [37]:
torch.cuda.empty_cache()